In [1]:
import cv2
import numpy as np
import pandas as pd
import random
import matplotlib.colors as mcolors
import sys
# import matplotlib.pyplot as plt
# %matplotlib inline

random.seed(0)

In [2]:
aruco = cv2.aruco
dictionary = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)

In [3]:
def getCornersFromID(_corners, _ids, _id):
    if not np.any(_ids == [_id]):    #指定されたidがidsに存在しなければFalseを返す
        return np.nan
    index = np.where(_ids == _id)[0][0] #whereは[[1次元目のindex], [2次元目のindex]] で返ってくるが、ほしいのは1次元目のindexだけなので[0][0]で指定
    return tuple(map(lambda c: tuple(c),_corners[index][0]))    #_corners[index][0][0~3]にそれぞれ座標が入っているのでタプルにして返す

def getCenterPoint(corners):
    # cornersがFalseの場合はFalseを返す（TODO:指定の座標いがはすべてFalseを返すように改良）
    # Trueであれば、左上座標と右下座標を平均した値を返す
    if type(corners) != tuple or np.isnan(corners[0][0]) or np.isnan(corners[2][0]) or np.isnan(corners[0][1]) or np.isnan(corners[2][1]):
        return np.nan
    x = int((corners[0][0] + corners[2][0]) / 2)
    y = int((corners[0][1] + corners[2][1]) / 2)
    return (x,y)


In [4]:
# シャープマスク（鮮鋭化）用のカーネル生成
def make_sharp_kernel(k: int):
  return np.array([
    [-k / 9, -k / 9, -k / 9],
    [-k / 9, 1 + 8 * k / 9, k / 9],
    [-k / 9, -k / 9, -k / 9]
  ], np.float32)

In [5]:
cap = cv2.VideoCapture("src/mov/test05.mp4")

if not (cap.isOpened()) :   # 正常に読み込めなかった場合終了する（VideoCaptureコンストラクタ自体は失敗してもFalseを返さないので注意）
    sys.exit()
fps = int(cap.get(cv2.CAP_PROP_FPS))
print("FPS: " + str(fps))
frameCount = 1
ret, frame = cap.read()
h, w = frame.shape[:2]

FPS: 29


In [6]:
cv2.startWindowThread()
corners, ids, rejectedImgPoints = aruco.detectMarkers(frame, dictionary)
# matplotlibのカラーテーブルを持ってくる（148色分）
colors = list(map(lambda color: tuple(map(lambda c: int(c * 255),mcolors.to_rgb(color))), mcolors.CSS4_COLORS.values()))
random.shuffle(colors)
orbit_length = 100

# 時間計測
timer_wait = cv2.TickMeter()
timer_fps = cv2.TickMeter()
timer_wait.start()
timer_fps.start()
while True:
    timer_wait.stop()
    timer_fps.stop()
    time_wait = timer_wait.getTimeMilli()
    time_fps = timer_fps.getTimeMilli()
    wait_time = max(1, int((1000 / fps) - time_wait))
    current_fps = 1000 / time_fps
    timer_fps.reset()
    timer_fps.start()
    ### ディレイ＆escキー、フレーム終端チェック
    if not ret or cv2.waitKey(wait_time) == 27:
        break
    timer_wait.reset()
    timer_wait.start()


    # print("frame: " + str(frameCount))
    # print("  timer: " + str(time_wait))
    # print("  wait : " + str(wait_time))
    # print("  fps: " + str(current_fps))
    print(current_fps)
    
    ### マーカー検出
    corners, ids, rejectedImgPoints = aruco.detectMarkers(frame, dictionary)

    ## idsを次元削減したリスト作成
    # 検出id数が1になるとmapのイテレータが回せなくなるため、最後にNaNを追加（あまりやりたくはない）
    id_list = np.append(np.squeeze(ids).copy(), np.nan)

    # 各マーカーの中心座標を求める
    # 最後の[:-1]でid_listに追加したnanの探索結果を除外
    centers = list(map(lambda id: getCenterPoint(getCornersFromID(corners, ids, id)), id_list))[:-1]

    # centersと同様にid_listも最後にNaNが入っているので除外
    id_list = id_list[:-1]

    # print(centers)
    if id_list[0] == None:  # マーカーが1つも検知できなかった場合
        orbit = pd.DataFrame([np.nan], index=[frameCount])  # すべてのcolがnanのDF作成
    else:
        orbit = pd.DataFrame([centers], columns=id_list, index=[frameCount])    # indexがフレーム数、col名が各id、値がセンター位置のDF作成

    if frameCount == 1: # 最初のフレームの場合、orbitで初期化
        orbits = orbit   
    else:               # 2フレーム目以降はorbitsにorbitを連結
        orbits =  pd.concat([orbits, orbit])

    ### 画面情報書き込み
    ## マーカーの枠とidの書き込み
    aruco.drawDetectedMarkers(frame, corners, ids, (255, 255, 0))

    ## 軌跡の書き込み
    for id, positions in orbits.iteritems():
        id = int(id)
        # print(positions)
        # 欠損値を補完
        # centersの座標をx, yのみ抽出→interpolateでnanをスプライン補完

        # x(i==0)、y(i==1)に関しての座標履歴を取得
        for i in np.arange(2):
            # print("i :" + str(i))
            
            # x，y座標の履歴をSeriesで作成
            comps = pd.Series(list(map(lambda pos: pos[i] if type(pos) == tuple else pos,positions)))
            # print("befor fill num comps ")
            # print(comps)
            # print("first elm: " + str(comps[0]))
            # 欠損値を補完（欠損値の前の値で補完したあと、欠損値のあとの値で補完）
            comps = comps.interpolate("ffill").interpolate("bfill").astype('int')
            if i == 0:
                xs = comps
            else:
                ys = comps
            # print("after fill num comps ")
            # print(comps)
            
        # 補完したx, y座標を元の(x, y)座標に結合し、末尾orbit_length個分の座標だけ代入
        positions = np.array(list(zip(xs, ys))[-orbit_length:])

        # positions = np.array(positions.to_list()[-30:])
        # print("positions")
        # print(positions)
        # print("id: " + str(id))
        # print("-----")
        cv2.polylines(frame, [positions],False, colors[id], 1)

    #### フレーム描画
    cv2.imshow("mov", frame)
    #### 次フレーム読み込み処理
    ret, frame = cap.read()
    frameCount += 1

cv2.destroyAllWindows()
cap.release()

orbits.to_csv("output/orbits.csv")

2515.0905432595573
8.544153193249095
47.71333826371162
30.333179645223126
19.481746577544172
24.641770264775822
20.5041141505043
15.88161211068213
40.85751757894694
17.983929560544695
55.5524693072607
30.652656359200087
21.119993241602163
20.906506104699783
22.301466767469297
23.10354547008784
18.4002796842512
23.79281219143697
23.277521595720657
20.044418431243635
21.91780821917808
21.49465213054992
21.537845224736646
22.615480296262792
22.591160982354044
18.89248475848792
22.988347206800874
22.407507411283078
21.172310851867927
22.136725269957363
22.383132071670786
19.596431097968438
22.722831446580894
22.195045621916275
13.453083715849345
39.93689969847641
21.290411863017493
21.92655481200172
21.2961247441803
22.546033363620168
21.16182660422517
16.42168253274894
30.291831504716438
21.805019515492468
19.673421207948063
24.77584058233136
21.25818705929121
21.903262052817524
20.0944842650141
24.282748321455024
22.176046764847417
22.19435597527549
21.15054748192157
23.125130078856696
2